<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/Classificazione_in_localizzabile_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usiamo ResNet50 per classificare le immagini in localizzabili oppure no

In [1]:
import os
import sys
from pathlib import Path

# with this line we can check if we are in colab or not
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)

cwd = Path(os.getcwd())
if in_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    os.chdir('/gdrive/MyDrive/GeoEstimation')
    
else:
    #our defult wd in local should be Vision_Project
    if str(cwd)[-14:] == 'Vision_Project':
        os.chdir(cwd / 'GeoEstimation')
    sys.path.append(cwd / 'GeoEstimation')

are we in Colab?: True
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
✨🍰✨ Everything looks OK!


In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import sys
from pathlib import Path
from PIL import Image
from numpy import asarray
import multiprocessing as mp

In [3]:
dataset_test = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/im2gps3k_places365_classified.csv").dropna()

In [4]:
# 4k dataset for the training
dataset = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/yfcc4k.csv").dropna()
indexNames = dataset[ dataset['LOC'] == 7 ].index
# Delete these row indexes from dataFrame
dataset.drop(indexNames , inplace=True)
wrong_row = dataset[dataset['IMG_ID'] == '2902900480(1).jpg'].index
dataset.drop(wrong_row, inplace=True)
set(dataset['LOC'])

{0.0, 1.0, 2.0}

In [15]:
images_test = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]

In [16]:
# 4k dataset for the training
images = ["/gdrive/MyDrive/GeoEstimation/resources/images/yfcc4k/{}".format(img) for img in dataset['IMG_ID']]

In [ ]:
#images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]
#img_all = [Image.open(image) for image in images]
#img_data = [asarray(img) for img in img_all]

In [20]:
# a function to load all the images in an array format in img_array
# other "easier solutions" were really slow, this works fine
import tensorflow as tf
from tqdm import tqdm
tf.config.set_visible_devices([], 'GPU')

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.resize((256, 256))
        return np.asarray(img)

# Crea una pool di processi
with mp.Pool() as pool:
  # Carica le immagini in parallelo
  images = tqdm(["/gdrive/MyDrive/GeoEstimation/resources/images/yfcc4k/{}".format(img) for img in dataset['IMG_ID']])
  img_data = pool.map(load_image, images)

# Crea un array multidimensionale a partire dalla lista di array numpy
img_array = np.stack(img_data)


100%|██████████| 4368/4368 [00:27<00:00, 161.01it/s]


In [ ]:
np.save(fout.npy, img_array)

In [42]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images4k_array.npy'
np.save(fout, img_array)

In [5]:
# load
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images4k_array.npy'
img_array = np.load(fout)

In [23]:
# a function to load all the images in an array format in img_array
# other "easier solutions" were really slow, this works fine

# test set

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

# Crea una pool di processi
with mp.Pool() as pool:
  # Carica le immagini in parallelo
  images_test = tqdm(["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset_test['IMG_ID']])
  img_data_test = pool.map(load_image, images_test)

# Crea un array multidimensionale a partire dalla lista di array numpy
img_array_test = np.stack(img_data_test)

100%|██████████| 2997/2997 [03:35<00:00, 13.88it/s]


In [43]:
fout2 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images_test3k_array.npy'
np.save(fout2, img_array_test)

In [6]:
# load
fout2 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images_test3k_array.npy'
img_array_test = np.load(fout2)

In [7]:
# now we can re-set the GPU
import tensorflow as tf
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [8]:
# the labels
label_array = np.array(dataset['LOC']).reshape((img_array.shape[0], 1))
# the labels
label_array_test = np.array(dataset_test['LOC']).reshape((img_array_test.shape[0], 1))

In [29]:
labels_int = label_array.astype(int)

# Conto gli zeri, gli uni e i due nell'array
count = np.bincount(labels_int.flatten())

# Stampo i risultati
print(f"Zeri: {count[0]}")
print(f"Uni: {count[1]}")
print(f"Due: {count[2]}")

Zeri: 1821
Uni: 768
Due: 1779


In [ ]:
# train-test split
# X_train, X_test, y_train, y_test = train_test_split(img_array, label_array, test_size=0.16, random_state=123)

In [18]:
X_train = img_array
y_train = label_array
X_test = img_array_test
y_test = label_array_test

In [28]:
# processing data for one-hot encoding
def preprocess_data(X, Y):
  X_p = preprocess_input(X)
  Y_p = to_categorical(Y, 3)
  return X_p, Y_p

In [19]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
y_test = enc.fit_transform(y_test).toarray()
y_train = enc.fit_transform(y_train).toarray()

In [23]:
# this
y_train = np.where(label_array != 0, 1, 0)
y_test = np.where(label_array_test != 0, 1, 0)

In [ ]:
# one hot encoding applied both to train and test sets
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)

In [9]:
# the model
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)

In [10]:
# set all the layers not-trainable
for layer in res_model.layers[:]:
  layer.trainable = False

In [11]:
# add trainable layers to performe the classification
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d (AverageP  (None, 7, 7, 2048)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dropout (Dropout)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 100,353
Non-trainable params: 23,587,712
______________________________________

In [ ]:
pip install pyyaml h5py

In [13]:
# checkpoints
checkpoint_path = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [64]:
# checkpoints and fit
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(X_train, y_train, batch_size=32, epochs=8, callbacks=[cp_callback], verbose=1)
model.summary()

Epoch 1/8
137/137 [==============================] - ETA: 0s - loss: 66.3989 - accuracy: 0.7626
Epoch 1: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
137/137 [==============================] - 21s 129ms/step - loss: 66.3989 - accuracy: 0.7626
Epoch 2/8
137/137 [==============================] - ETA: 0s - loss: 29.7563 - accuracy: 0.8498
Epoch 2: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
137/137 [==============================] - 18s 128ms/step - loss: 29.7563 - accuracy: 0.8498
Epoch 3/8
137/137 [==============================] - ETA: 0s - loss: 15.4552 - accuracy: 0.9009
Epoch 3: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
137/137 [==============================] - 17s 125ms/step - loss: 15.4552 - accuracy: 0.9009
Epoch 4/8
137/137 [==============================] - ETA: 0s - loss: 13.9395 - accuracy: 0.9041
Epoch 4: saving model to /gdrive/MyDrive/GeoEstim

In [27]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

In [65]:
# test
test_loss, test_acc = model.evaluate(X_test, y_test)

94/94 [==============================] - 12s 117ms/step - loss: 53.7093 - accuracy: 0.8208


In [55]:
from sklearn.metrics import plot_roc_curve

In [54]:
model.predict(X_test)

94/94 [==============================] - 12s 120ms/step


array([[1.],
       [1.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

## 4k + 3k as train set

In [14]:
X = np.concatenate((img_array, img_array_test))
y = np.concatenate((label_array, label_array_test))

In [15]:
# this
y = np.where(y != 0, 1, 0)

In [16]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.16, 
                                                    random_state=123)

In [17]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[cp_callback], verbose=1)
model.summary()

Epoch 1/10
194/194 [==============================] - ETA: 0s - loss: 60.8832 - accuracy: 0.7627
Epoch 1: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
194/194 [==============================] - 36s 125ms/step - loss: 60.8832 - accuracy: 0.7627
Epoch 2/10
193/194 [============================>.] - ETA: 0s - loss: 34.9793 - accuracy: 0.8347
Epoch 2: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
194/194 [==============================] - 24s 121ms/step - loss: 34.9900 - accuracy: 0.8343
Epoch 3/10
193/194 [============================>.] - ETA: 0s - loss: 25.9774 - accuracy: 0.8640
Epoch 3: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
194/194 [==============================] - 24s 123ms/step - loss: 26.0041 - accuracy: 0.8640
Epoch 4/10
193/194 [============================>.] - ETA: 0s - loss: 21.4230 - accuracy: 0.8812
Epoch 4: saving model to /gdrive/MyDrive/GeoE

In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test)

37/37 [==============================] - 6s 138ms/step - loss: 39.7302 - accuracy: 0.8338


### load model

In [19]:
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d_1 (Averag  (None, 7, 7, 2048)       0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_1 (Dropout)         (None, 100352)            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 23,634,945
Non-trainable params: 53,120
_____________________________________

In [20]:
model.load_weights(checkpoint_path)

In [21]:
test_loss, test_acc = model.evaluate(X_test, y_test)

37/37 [==============================] - 7s 118ms/step - loss: 39.7302 - accuracy: 0.8338


In [23]:
model.predict(X_test).round()

37/37 [==============================] - 4s 116ms/step


array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

## Predict on the 22k

In [ ]:
#dataset_25k = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/data25k_places365.csv").dropna()

In [ ]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/img_array22k.npy'
img_array25k = np.load(fout)

In [ ]:
import tensorflow as tf
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [ ]:
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint_path = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt'

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
y_pred25k = model.predict(img_array25k)

In [ ]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/y_pred25k.npy'
np.save(fout, y_pred25k)